# Free Recall Memory Experiment Analysis

This notebook analyzes data from a free recall memory experiment to understand:
- Overall memory performance
- Serial position effects (primacy and recency)
- Individual trial performance

## Background Theory
- **Primacy Effect**: Better recall for words at the beginning of the list (long-term memory encoding)
- **Recency Effect**: Better recall for words at the end of the list (working memory)
- **Serial Position Curve**: The U-shaped curve showing recall probability by position

## Import Required Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

# Set up plotting style
plt.style.use('default')
plt.rcParams['figure.figsize'] = (12, 8)

## Load and Prepare Data

In [ ]:
# Load the data
data_file = 'data/free_recall_results.csv'
df = pd.read_csv(data_file)

print("Dataset Overview:")
print(f"Number of trials: {len(df)}")
print(f"Columns: {list(df.columns)}")
print("\nFirst few rows:")
df.head()

In [ ]:
def parse_words(word_string):
    """Convert '[word1, word2, word3]' into a clean list of words"""
    if pd.isna(word_string) or word_string == '[]':
        return []
    
    # Remove brackets and quotes, then split by comma
    cleaned = word_string.strip('[]').replace('"', '').replace("'", '')
    words = [word.strip() for word in cleaned.split(',') if word.strip()]
    return words

# Test the function
sample_string = df.iloc[0]['presented_words']
print(f"Original: {sample_string}")
print(f"Parsed: {parse_words(sample_string)}")

## Calculate Performance Metrics

In [ ]:
# Calculate accuracy for each trial
all_accuracies = []
trial_details = []

print("Trial-by-Trial Analysis:")
print("-" * 40)

for _, row in df.iterrows():
    trial_num = row['trial']
    
    # Parse the words
    presented_words = parse_words(row['presented_words'])
    recalled_words = parse_words(row['recalled_words'])
    
    # Calculate accuracy
    presented_set = set(presented_words)
    recalled_set = set(recalled_words)
    correct_words = presented_set.intersection(recalled_set)
    
    accuracy = len(correct_words) / len(presented_words) if len(presented_words) > 0 else 0
    all_accuracies.append(accuracy)
    
    trial_details.append({
        'trial': trial_num,
        'presented': presented_words,
        'recalled': recalled_words,
        'correct': list(correct_words),
        'accuracy': accuracy,
        'n_presented': len(presented_words),
        'n_recalled': len(recalled_words),
        'n_correct': len(correct_words)
    })
    
    print(f"Trial {trial_num}: {len(correct_words)}/{len(presented_words)} = {accuracy:.1%}")

print(f"\nOverall Statistics:")
print(f"Average accuracy: {np.mean(all_accuracies):.1%}")
print(f"Best performance: {max(all_accuracies):.1%}")
print(f"Worst performance: {min(all_accuracies):.1%}")
print(f"Standard deviation: {np.std(all_accuracies):.1%}")

## Serial Position Analysis

In [ ]:
# Analyze which positions are remembered best
position_correct = {}

for trial in trial_details:
    presented = trial['presented']
    recalled = trial['recalled']
    
    # Track which positions were remembered correctly
    for i, word in enumerate(presented):
        position = i + 1  # Start counting from 1
        if position not in position_correct:
            position_correct[position] = []
        
        # Was this word remembered?
        was_remembered = word in recalled
        position_correct[position].append(was_remembered)

# Calculate recall probability for each position
position_probabilities = {}
print("Position Effects:")
print("-" * 20)

for pos in sorted(position_correct.keys()):
    prob = np.mean(position_correct[pos])
    position_probabilities[pos] = prob
    print(f"Position {pos}: {prob:.1%}")

# Calculate primacy and recency effects
positions = sorted(position_probabilities.keys())
if len(positions) >= 5:
    primacy_positions = positions[:3]
    recency_positions = positions[-3:]
    middle_positions = positions[3:-3] if len(positions) > 6 else []
    
    primacy_prob = np.mean([position_probabilities[pos] for pos in primacy_positions])
    recency_prob = np.mean([position_probabilities[pos] for pos in recency_positions])
    middle_prob = np.mean([position_probabilities[pos] for pos in middle_positions]) if middle_positions else 0
    
    print(f"\nMemory Effects:")
    print(f"Primacy Effect (first 3): {primacy_prob:.1%}")
    if middle_positions:
        print(f"Middle positions: {middle_prob:.1%}")
    print(f"Recency Effect (last 3): {recency_prob:.1%}")
    
    if primacy_prob > middle_prob + 0.1:
        print("Primacy effect detected!")
    if recency_prob > middle_prob + 0.1:
        print("Recency effect detected!")

## Visualizations

In [ ]:
# Create comprehensive plots
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# 1. Serial Position Curve
positions_list = list(position_probabilities.keys())
probabilities_list = list(position_probabilities.values())

axes[0,0].plot(positions_list, probabilities_list, 'bo-', linewidth=2, markersize=8)
axes[0,0].set_xlabel('Word Position in List')
axes[0,0].set_ylabel('Recall Probability')
axes[0,0].set_title('Serial Position Curve')
axes[0,0].grid(True, alpha=0.3)
axes[0,0].set_ylim(0, 1)

# Add colored regions for primacy/recency
if len(positions_list) >= 5:
    axes[0,0].axvspan(1, 3, alpha=0.2, color='blue', label='Primacy')
    axes[0,0].axvspan(max(positions_list)-2, max(positions_list), alpha=0.2, color='red', label='Recency')
    axes[0,0].legend()

# 2. Trial Performance
trial_numbers = [t['trial'] for t in trial_details]
trial_accuracies = [t['accuracy'] for t in trial_details]

axes[0,1].plot(trial_numbers, trial_accuracies, 'go-', linewidth=2, markersize=6)
axes[0,1].set_xlabel('Trial Number')
axes[0,1].set_ylabel('Accuracy')
axes[0,1].set_title('Performance Across Trials')
axes[0,1].grid(True, alpha=0.3)
axes[0,1].set_ylim(0, 1)

# 3. Accuracy Distribution
axes[1,0].hist(all_accuracies, bins=8, alpha=0.7, color='skyblue', edgecolor='black')
axes[1,0].set_xlabel('Accuracy')
axes[1,0].set_ylabel('Number of Trials')
axes[1,0].set_title('Distribution of Accuracy Scores')
axes[1,0].axvline(np.mean(all_accuracies), color='red', linestyle='--', 
                 label=f'Mean: {np.mean(all_accuracies):.1%}')
axes[1,0].legend()

# 4. Memory Effects Comparison
if len(positions_list) >= 5:
    categories = ['First\n(Primacy)', 'Middle', 'Last\n(Recency)']
    values = [primacy_prob, middle_prob, recency_prob]
    colors = ['lightblue', 'lightgray', 'lightcoral']
    
    bars = axes[1,1].bar(categories, values, color=colors, alpha=0.7, edgecolor='black')
    axes[1,1].set_ylabel('Recall Probability')
    axes[1,1].set_title('Memory Effects Comparison')
    axes[1,1].set_ylim(0, 1)
    
    # Add value labels on bars
    for bar, value in zip(bars, values):
        axes[1,1].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.02, 
                      f'{value:.1%}', ha='center', fontweight='bold')

plt.tight_layout()
plt.show()

## Summary and Conclusions

In [ ]:
# Generate final summary
print("FREE RECALL EXPERIMENT SUMMARY")
print("=" * 40)
print(f"Total trials analyzed: {len(df)}")
print(f"Average accuracy: {np.mean(all_accuracies):.1%}")
print(f"Performance range: {min(all_accuracies):.1%} - {max(all_accuracies):.1%}")

if len(positions_list) >= 5:
    print(f"\nSerial Position Effects:")
    print(f"Primacy effect (first 3 positions): {primacy_prob:.1%}")
    print(f"Recency effect (last 3 positions): {recency_prob:.1%}")
    if middle_positions:
        print(f"Middle positions: {middle_prob:.1%}")
    
    print(f"\nConclusions:")
    if primacy_prob > middle_prob + 0.1:
        print("- Primacy effect confirmed: Long-term memory encoding of early items")
    if recency_prob > middle_prob + 0.1:
        print("- Recency effect confirmed: Working memory retention of recent items")
    
    if primacy_prob > middle_prob + 0.1 and recency_prob > middle_prob + 0.1:
        print("- Classic U-shaped serial position curve observed")
    
print(f"\nThis demonstrates the dual-store model of memory:")
print("- Early items → Long-term memory (primacy)")
print("- Recent items → Working memory (recency)")
print("- Middle items → Neither effect, lower recall")